In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium
from tqdm import tqdm



In [2]:
# Replace with the correct filename
df = pd.read_csv("Kwifi.csv", encoding="windows-1252")

# Preview the data
df.head()



,SL NO,District,Location Name As Per List,Location Address,Location Category,Hotspot included in Phase
0,1,Thiruvananthapuram,Akkulam childrens park,"Aluvila Rd,Akkulam,Tvm,kerala 695011",Tourist Place,Phase-2
1,2,Thiruvananthapuram,"Amboori Panchayath Office,","amboori neyyar road, Amboori, Kerala 695505",Panchayat office,Phase-1
2,3,Thiruvananthapuram,Anad Panchayath Office,"Anad, Kerala 695541 / Post office at Anad Bank...",Panchayat office,Phase-1
3,4,Thiruvananthapuram,Andoorkonam Panchayath Office,"Kaniyapuram, Kerala 695301",Panchayat office,Phase-1
4,5,Thiruvananthapuram,APJ Abdul Kalam Technological University,APJ Abdul Kalam Technological University CET C...,University/College,Phase-2


### in below cell select the require data and try to redure it for faster process 

In [3]:
print(df.columns)

Index(['SL NO', 'District', 'Location Name As Per List', 'Location Address',
       'Location Category', 'Hotspot included in Phase '],
      dtype='object')


In [4]:
df = df.head(50) # i want first 100 rows for testing purpose

In [5]:
# Combine with state context for better geocoding
df["Location"] = df["Location Address"].astype(str) + ", Kerala, India"

# Set up the geocoder with a user agent and rate limiter
geolocator = Nominatim(user_agent="kerala_wifi_mapper")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Use tqdm for progress bar in notebook
tqdm.pandas()


In [6]:
def safe_geocode(address):
    try:
        return geocode(address)
    except Exception as e:
        print(f"Geocoding error for {address}: {e}")
        return None

# Apply geocoding
df["Coordinates"] = df["Location"].progress_apply(safe_geocode)

# Extract latitude and longitude
df["Latitude"] = df["Coordinates"].apply(lambda loc: loc.latitude if loc else None)
df["Longitude"] = df["Coordinates"].apply(lambda loc: loc.longitude if loc else None)

# Drop rows where geocoding failed
df = df.dropna(subset=["Latitude", "Longitude"])

# Preview processed data
df[["Location Name As Per List", "Location", "Latitude", "Longitude"]].head()


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:53<00:00,  1.08s/it]


,Location Name As Per List,Location,Latitude,Longitude
3,Andoorkonam Panchayath Office,"Kaniyapuram, Kerala 695301, Kerala, India",8.587665,76.856070
10,Athiyanoor block panchayath office,"Aralumoodu, 695123, Kerala, India",8.411612,77.053500
11,Attingal,"Attingal Municipality, Kerala 695101, Kerala, ...",8.696762,76.816301
12,Attingal Bus Stand,"Attingal, Kerala 695101, Kerala, India",8.698597,76.813440
13,Attingal Munsif court,"NH 47, Attingal, Kerala 695101, Kerala, India",8.698597,76.813440


In [7]:
# Create base map centered over Kerala
wifi_map = folium.Map(location=[10.8505, 76.2711], zoom_start=7)

# Add markers to the map
for _, row in df.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=row["Location Name As Per List"],
        tooltip=row["Location Category"]
    ).add_to(wifi_map)

# Display the map directly in the notebook
wifi_map


In [8]:
# Save the map as an interactive HTML file
wifi_map.save("kerala_wifi_map_100.html")
print("Map saved to kerala_wifi_map_100.html")


Map saved to kerala_wifi_map_100.html
